In [24]:
%matplotlib inline
import pandas as pd
import pandas_profiling as pp
import numpy as np # linear algebra
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MultiLabelBinarizer 
import seaborn as sns
import tkinter
import matplotlib
import matplotlib.colors
import matplotlib.pyplot as plt

In [25]:
data = pd.read_csv('final_dataset.csv')
colors_amazon = ['#0F79AF', '212529', '343A40']
color_netflix = ['#E50914']
colors = ['#fca311', ]

# What kind of content is present on Streaming Platforms? 

### Tv Series/Movie distribution sunburst
First we plot in a sunbarst the distribution of content typology in our dataset (Movie, TV Show) and the consequent subdivision between Amazon and Netflix.

In [41]:
x= data.groupby(['type'])[['amazon', 'netflix']].sum()
y=len(data)
r=((x*100/y)).round(2)
ratio = pd.DataFrame(r)
ratio
df = ratio.stack().reset_index()
df.columns = ['type','distr','val']
df['distr'] = df['distr'].apply(lambda x: x.capitalize())
df

,type,distr,val
0,Movie,Amazon,35.80
1,Movie,Netflix,43.78
2,TV Show,Amazon,5.85
3,TV Show,Netflix,16.37


In [43]:
fig = px.sunburst(df, path=['distr', 'type'], values='val',color='distr', hover_name="type", color_discrete_map={'Netflix':'#E50914', 'Amazon':'#0F79AF'})
fig.update_traces(marker=dict(line=dict(color='black', width=2)), textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))
fig.update_traces(hovertemplate=('<br>%{customdata[0]} presence is: %{value}%<br>'))
fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)' )
fig.show()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/plotly/express/_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



**conclusion:** Netflix is more represented in our dataset than Movies. Surprisingly there are more films than TV series in both platforms

### Country analysis

In [45]:
df = data.copy()
df['country'].replace('no_data', np.nan, inplace  = True)
df.dropna(inplace=True)
df['count'] = 1
df['first_country'] = df['country'].apply(lambda x: x.split(",")[0])
df['first_country'].replace('United States', 'USA', inplace=True)
df['first_country'].replace('United Kingdom', 'UK',inplace=True)
df['first_country'].replace('South Korea', 'S. Korea',inplace=True)
x = df.groupby('first_country')['count'].sum().sort_values(ascending=False)[0:10]
y=len(df)
r=((x*100/y)).round(2)
df = pd.DataFrame(r)
df

,count
first_country,
USA,45.20
India,16.13
UK,8.64
Canada,3.89
France,2.34
S. Korea,2.04
Spain,1.74
Australia,1.73
Germany,1.28


In [49]:
fig = go.Figure(data=[go.Bar(x=df.index, y=df['count'].to_list(),
            hovertext='total ammount')])
fig.update_traces(marker_color='#fca311', marker_line_color='black',
                  marker_line_width=2)
fig.update_traces(marker=dict(line=dict(color='black', width=2)), textfont=dict(family="Helvetica",size=16,color="black"),hoverlabel=dict(font_size=14,font_family="Helvetica"))
fig.update_layout(paper_bgcolor="#e1e1e1", plot_bgcolor = 'rgba(0, 0, 0, 0)' )
fig.show()